In [3]:
!pip install fsspec

In [4]:
!pip install pyarrow

In [5]:
!pip install s3fs

In [1]:
!pip install dask[complete] fsspec s3fs pyarrow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 7.4 MB/s eta 0:00:00
  Attempting uninstall: botocore
    Found existing installation: botocore 1.37.38
    Uninstalling botocore-1.37.38:
      Successfully uninstalled botocore-1.37.38
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
s3transfer 0.11.5 requires botocore<2.0a.0,>=1.37.4, but you have botocore 1.37.3 which is incompatible.
boto3 1.37.34 requires botocore<1.38.0,>=1.37.34, but you have botocore 1.37.3 which is incompatible.


In [7]:
!curl -O https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2019-01.parquet

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  105M  100  105M    0     0  78.7M      0  0:00:01  0:00:01 --:--:-- 78.7M


In [2]:
import dask.dataframe as dd

# ddf = dd.read_parquet(
#     "s3://nyc-tlc/trip-data/",
#     storage_options={"anon":True},
#     engine="pyarrow"
# )

dff = dd.read_parquet("yellow_tripdata_2019-01.parquet")

print(dff.columns)
# print(dff.head(5))

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')


In [3]:
#obliczaenie średniej odległości przejazdu
mean_distance = dff["trip_distance"].mean().compute()
print(f"Średnia odległość przejzdu: {mean_distance:.2f} mil")

Średnia odległość przejzdu: 2.83 mil


In [4]:
#liczba przejzdów dziennie
dff["tpep_dropoff_datetime"] = dd.to_datetime(dff["tpep_dropoff_datetime"])

#grupowanie po dacie i liczenie liczby przejazdów
daily_counts = dff.groupby(dff["tpep_dropoff_datetime"].dt.date).size().compute()
print(daily_counts)

tpep_dropoff_datetime
2001-02-02         1
2003-01-01         2
2008-12-31        16
2009-01-01        56
2018-11-09         1
2018-11-28         9
2018-11-29         2
2018-12-21         1
2018-12-25         1
2018-12-30         7
2018-12-31       220
2019-01-01    188089
2019-01-02    198635
2019-01-03    223634
2019-01-04    235350
2019-01-05    236110
2019-01-06    210247
2019-01-07    228971
2019-01-08    237042
2019-01-09    255442
2019-01-10    281348
2019-01-11    290863
2019-01-12    265039
2019-01-13    229658
2019-01-14    244904
2019-01-15    267157
2019-01-16    272384
2019-01-17    284154
2019-01-18    266260
2019-01-19    236800
2019-01-20    204113
2019-01-21    193013
2019-01-22    254922
2019-01-23    260830
2019-01-24    281394
2019-01-25    291233
2019-01-26    272094
2019-01-27    222681
2019-01-28    241106
2019-01-29    259408
2019-01-30    276534
2019-01-31    283867
2019-02-01      2986
2019-02-02         3
2019-02-04         2
2019-02-13         1
2019-02-24  

In [5]:
import glob

files = glob.glob("yellow_tripdata_2019-01.parquet")

ddf = dd.read_parquet(files)

mean_distance = ddf["trip_distance"].mean().compute()
print(f"Średnia odległość przejzadu: {mean_distance:.2f} mil")

Średnia odległość przejzadu: 2.83 mil
